# Блок №7

# Задание 1
___
Ваш менее опытный коллега не уверен, что его таблица `retail` действительно создалась и добавилась в базу данных. Напишите запрос, который создаст таблицу, если она еще не существует (`IF NOT EXISTS`), со следующими параметрами:

+ **Название**: `retail`
+ **База**: `test`
+ **Столбцы**:
 
 - `InvoiceNo` – идентификатор заказа, строковое значение
 - `StockCode` – идентификатор товара, строковое значение
 - `Description` – описание товара, строковое значение
 - `Quantity` – количество единиц товара, добавленных в корзину (особенность данных – значения могут быть отрицательными, 32 бита)
 - `InvoiceDate` – дата и время создания заказа (транзакции), часовой пояс – 'Europe/London'
 - `UnitPrice` – цена за единицу товара (особенность данных – значения могут быть отрицательными, точность – три знака после точки, 64 бита)
 - `CustomerID` – идентификатор пользователя, целое положительное число, 32 бита.
 - `Country` – страна, строковое значение. 

+ **Движок**: `MergeTree`
+ **Сортировка**: `InvoiceDate`, `CustomerID`

Правильный запрос:
```
CREATE TABLE IF NOT EXISTS test.retail ( 
    InvoiceNo String, 
    StockCode String, 
    Description String,
    Quantity Int32, 
    InvoiceDate DateTime('Europe/London'),
    UnitPrice Decimal64(3),
    CustomerID UInt32,
    Country String
) 
ENGINE = MergeTree 
ORDER BY (InvoiceDate, CustomerID)
```

# Задание 2
___
В следующих шагах используется `default.retail` – данные о транзакциях британского интернет-магазина в период с 1 декабря 2010 по 9 декабря 2011, где:

+ `InvoiceNo` – номер транзакции
+ `StockCode` – код товара
+ `Description` – описание товара
+ `Quantity` – количество единиц товара, добавленных в заказ
+ `InvoiceDate` – дата транзакции 
+ `UnitPrice` – цена за единицу товара
+ `CustomerID` – id клиента
+ `Country` – страна, где проживает клиент

Для начала посмотрите, в каких странах имеется наибольшее число уникальных пользователей. Постройте барчарт, где:

+ по оси x – страна
+ по оси y – число пользователей

В качестве ответа укажите число уникальных пользователей из Великобритании. 

Правильный запрос:
```
SELECT
    COUNT(DISTINCT CustomerID) as NumUsers,
    Country
FROM
    default.retail
GROUP BY 
    Country
ORDER BY 
    NumUsers DESC
LIMIT 10
```
Ответ:
```
3950
```

Теперь добавим визуализацию в виде Chart, указав ось X как Country и NumUsers как ось Y:
[ссылка на график](http://redash.lab.karpov.courses/embed/query/5138/visualization/15893?api_key=PvQE2XdAT4be8pAd5EshhqHfiFmCCLksh8J7VbKd&)
![](https://ucarecdn.com/0364c873-6d76-403e-8743-eed83bdbfc73/)

# Задание 3
___
Следующая задача – посмотреть на динамику изменения числа активных пользователей в месяц в Великобритании, Австралии и Нидерландах. Полученная вами результирующая таблица должна иметь вид: страна - число уникальных пользователей за февраль 2011.

+ MAU (monthly active users) – число уникальных пользователей за месяц. Активные пользователи – те, кто сделал хотя бы один заказ за выбранный промежуток времени (месяц).

В качестве ответа укажите наименьшее число пользователей за февраль 2011. 

Правильный запрос:
```
SELECT
    COUNT(DISTINCT CustomerID) as act_users,
    Country,
    toStartOfMonth(InvoiceDate) as so_month
FROM
    retail
WHERE 
    Country IN ('United Kingdom', 'Netherlands', 'Australia')
GROUP BY 
    Country, so_month
ORDER BY
    Country, so_month
```
Ответ:
```
2
```

Добавим визуализацию: поменяем тип Chart на Line, по оси X - месяц, по оси Y - количество уникальных пользователей, Group By по стране. \
[ссылка на график](http://redash.lab.karpov.courses/embed/query/5141/visualization/10912?api_key=kPz7oA4cigJi7goIFUU6zQEncMNUinsuuKIAK1N5&)
![](https://ucarecdn.com/84b2ec0c-d382-4006-bf55-d41d31e8c48b/)

# Задание 4
___
Как вы могли заметить, график с предыдущего шага получается не очень информативным для 2 из 3 стран. Давайте посмотрим на динамику изменения числа активных пользователей в месяц уже для всех стран, кроме Великобритании. 

В каких месяцах число активных пользователей во Франции было больше, чем в Германии?

Правильный запрос:
```
SELECT
    COUNT(DISTINCT CustomerID) as act_users,
    Country,
    toStartOfMonth(InvoiceDate) as so_month
FROM
    retail
WHERE 
    Country IN ('Germany', 'France')
GROUP BY 
    Country, so_month
ORDER BY
    Country, so_month
```
Ответ:
```
Сентябрь 2011
Май 2011
Декабрь 2011
```

Можно построить такой же лайнчарт для нового запроса и посчитать месяца, где линия с пользователями Франции выше, чем с пользователями Германии: \
[ссылка на график](http://redash.lab.karpov.courses/embed/query/5142/visualization/10916?api_key=e0320SEsoGfan8P2evEzg3isDbI75KNhpcpBpbLF&)
![](https://ucarecdn.com/3df7f7ba-d2f6-403f-994c-32b61af2084d/)

# Задание 5
___
Теперь проанализируем сами заказы. Посчитайте среднюю сумму заказа (AOV – average order value) в каждой из стран.

Обратите внимание, что пользователь может добавить несколько одинаковых товаров в один заказ, и для подсчета суммы каждого заказа можно создать столбец `TotalPrice`, в котором будет храниться цена, которую пользователь заплатил, например, за три упаковки чая.

| InvoiceNo | Description | Quantity | UnitPrice | TotalPrice |
|-----------|-------------|----------|-----------|------------|
| 12345 | 12345 | 3 | 1.2 | 3.6 |
|12345 | COOKIES | 1 | 2.5 | 2.5 |

Далее для каждого заказа нужно просуммировать полученную колонку, а затем сгруппировать данные по странам и посчитать среднюю сумму заказа в каждой из них.

Визуализируйте результат. Из представленных вариантов ответа выберите страны, входящие в топ-3 по величине средней суммы заказа.

В таблице есть отрицательные значения `Quantity`, что соответствует отмененным заказам. Их в расчет AOV включать не нужно.

Правильный запрос:
```
SELECT
    Country,
    AVG(sum_order) AS avg_order
FROM (
    SELECT
        Country,
        InvoiceNo,
        SUM(Quantity * UnitPrice) AS sum_order
    FROM 
        retail
    WHERE Quantity > 0
    GROUP BY
        Country, InvoiceNo
    )
GROUP BY
    Country
```
Ответ:
```
Netherlands
Singapore
Australia
```

Также можно сразу ограничить запрос тремя записями и задать сортировку по убыванию средней стоимости, чтобы получить ответ. \
[ссылка на график](http://redash.lab.karpov.courses/embed/query/5144/visualization/10918?api_key=F6uUv3fy4q9tUaeLC5MnnMaT4KMAqWk4IGG5ONjm&)
![](https://storage.yandexcloud.net/klms-public/production/learning-content/28/373/2894/8896/35196/image.png)

# Задание 6
___
Как изменялась средняя сумма заказа в разных странах по месяцам?

Предположим, нас интересует динамика в следующих странах: United Kingdom, Germany, France, Spain, Netherlands, Belgium, Switzerland, Portugal, Australia, USA. Визуализируйте результат и выберите верные утверждения.

Правильный запрос:
```
SELECT
    Country,
    order_month,
    AVG(sum_order) AS avg_order
FROM (
    SELECT
        Country,
        InvoiceNo,
        toStartOfMonth(InvoiceDate) as order_month,
        SUM(Quantity * UnitPrice) AS sum_order
    FROM 
        retail
    GROUP BY
        Country, InvoiceNo, order_month
    )
WHERE 
    Country IN ('United Kingdom', 'Germany', 'France', 'Spain', 'Netherlands', 'Belgium', 
    'Switzerland', 'Portugal', 'Australia', 'USA')
GROUP BY
    Country, order_month
ORDER BY
    Country
```
Ответ:
```
Самые большие значения среднего чека наблюдаются в Австралии и Нидерландах.
Аномальный рост среднего чека произошел в августе 2011 среди пользователей из Австралии
С июля/августа средний чек в Швейцарии снова начал расти
```

Полученный график позволит пройтись по всем предложенным утверждениям из списка и выбрать корректные. \
[ссылка на график](http://redash.lab.karpov.courses/embed/query/5145/visualization/10920?api_key=ahfv2X8I8LpMLkQ64bgqE5QmMngAMP2dZUtPWvJn&)
![](https://ucarecdn.com/68f55680-c127-4808-8825-8ae63efc17c6/)

# Задание 7
___
Сколько товаров пользователи обычно добавляют в корзину? Посчитайте среднее количество товаров, добавленных в корзину, с разбивкой по странам. 

В качестве ответа укажите среднее число товаров в заказе среди пользователей в Канаде (Canada).

Правильный запрос:
```
SELECT
    Country,
    AVG(ItemsInOrder) as AvgPerOrder
FROM (
    SELECT 
        Country,
        SUM(Quantity) as ItemsInOrder
    FROM
        retail
    GROUP BY
        InvoiceNo, 
        Country
    )
WHERE Country='Canada'
GROUP BY Country
```
Ответ:
```
460.50
```

# Задание 8
___
Возможно, результат на предыдущем шаге показался вам странным, особенно если соотнести средний размер корзины с числом уникальных пользователей в некоторых странах. 

Посмотрим на Нидерланды (Netherlands) более подробно. Сгруппируйте данные по пользователям и посмотрите, кто купил наибольшее число товаров. В ответе укажите идентификатор данного пользователя. 

Правильный запрос:
```
SELECT
    CustomerID,
    SUM(Quantity) as NumItems
FROM
    retail
WHERE 
    Country = 'Netherlands'
GROUP BY 
    CustomerID
ORDER BY
    NumItems DESC
LIMIT 1
```
Ответ:
```
14646
```

# Задание 9
___
Один из пользователей добавил к себе в корзину почти 200 тысяч товаров. Не кажется ли это подозрительным? Изучите, что именно он покупает, посмотрите на количество товаров в каждом заказе, итоговые суммы. Подумайте, кто или что это может быть 🤔

Когда подумаете, просто введите идентификатор данного пользователя еще раз в поле ниже.

Ответ:
```
14646
```

1) В качестве первого шага можно просто забрать все имеющиеся записи в retail по CustomerID этого пользователя, отсортировать результат по количеству купленного и посмотреть на таблицу глазами - есть ли что-то необычное, видны ли какие-либо закономерности?

2) Можно попробовать собрать самые массово купленные этим пользователем товары и разбить их на категории - посмотреть, что было куплено. Также можно попробовать поискать закономерности в датах покупок

3) Можно было бы проверить этот результат на фрод - обычно для всех транзакций в интернет-магазинах есть таблица с верификацией этих транзакций, по которой можно убедиться, были ли платежи реальными. Вероятно, если платежи реальны, это могла быть оптовая закупка - например, другим магазином, а если нет - то это мог быть фрод или боты. Также не исключено, что проблема на самом деле в данных - например, покупки от множества пользователей записались в базу под одним идентификатором.